In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [ ]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
])

In [ ]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                 StructField("driverRef", StringType(), True)
                                 StructField("number", IntegerType(), True)
                                 StructField("code", StringType(), True)
                                 StructField("name", name_schema, True)
                                 StructField("dob", DateType(), True)
                                 StructField("nationality", StringType(), True)
                                 StructField("url", StringType(), True)
])

In [ ]:
drivers_df = spark.read \
    .schema(drivers_schema) \
    .json("/mnt/formula1dl/raw/drivers.json")

In [ ]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [ ]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("ingestion_date", current_timestamp()) \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name_surname")))

In [ ]:
display(drivers_with_columns_df)

In [ ]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

In [ ]:
drivers_final_df.write.mode("overwrite").parquet("/mnt/formula1dl/procssed/drivers")

In [ ]:
display(spark.read.parquet("/mnt/formula1dl/processed/drivers"))